# Week 3
### Context

#### Ensemble
+ Voting Ensemble
+ Out-of-fold(OOF) Ensemble
+ Stacking Ensemble

In [1]:
import os
from os.path import join

import multiprocessing
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

n_cpus = multiprocessing.cpu_count()

In [2]:
BASE_DIR = './'

train_path = join(BASE_DIR, 'data', 'MDC14', 'train.csv')
test_path  = join(BASE_DIR, 'data', 'MDC14', 'test.csv')

data = pd.read_csv(train_path)
test = pd.read_csv(test_path)

label = data['credit']

In [3]:
data.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [4]:
data.shape

(26457, 20)

In [5]:
data.describe()

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit
count,26457.000000,26457.000000,2.645700e+04,26457.000000,26457.000000,26457.0,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000
mean,13228.000000,0.428658,1.873065e+05,-15958.053899,59068.750728,1.0,0.224742,0.294251,0.091280,2.196848,-26.123294,1.519560
std,7637.622372,0.747326,1.018784e+05,4201.589022,137475.427503,0.0,0.417420,0.455714,0.288013,0.916717,16.559550,0.702283
min,0.000000,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.000000,-60.000000,0.000000
25%,6614.000000,0.000000,1.215000e+05,-19431.000000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000,-39.000000,1.000000
50%,13228.000000,0.000000,1.575000e+05,-15547.000000,-1539.000000,1.0,0.000000,0.000000,0.000000,2.000000,-24.000000,2.000000
75%,19842.000000,1.000000,2.250000e+05,-12446.000000,-407.000000,1.0,0.000000,1.000000,0.000000,3.000000,-12.000000,2.000000
max,26456.000000,19.000000,1.575000e+06,-7705.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000,0.000000,2.000000


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

In [7]:
test.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month
0,26457,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,1,0,1,0,NaN,2.0,-60.0
1,26458,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,-18964,-8671,1,0,1,0,Core staff,2.0,-36.0
2,26459,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,-15887,-217,1,1,1,0,Laborers,2.0,-40.0
3,26460,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-19270,-2531,1,1,0,0,Drivers,2.0,-41.0
4,26461,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,-17822,-9385,1,1,0,0,Managers,2.0,-8.0


In [8]:
test.describe()

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month
count,10000.00000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,31456.50000,0.434700,1.850433e+05,-16020.466400,59776.690400,1.0,0.227600,0.296300,0.085600,2.202700,-26.272400
std,2886.89568,0.729102,1.015398e+05,4197.672887,138121.224504,0.0,0.419304,0.456648,0.279786,0.898272,16.348557
min,26457.00000,0.000000,2.700000e+04,-25152.000000,-15661.000000,1.0,0.000000,0.000000,0.000000,1.000000,-60.000000
25%,28956.75000,0.000000,1.215000e+05,-19483.250000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000,-39.000000
50%,31456.50000,0.000000,1.575000e+05,-15606.000000,-1577.000000,1.0,0.000000,0.000000,0.000000,2.000000,-25.000000
75%,33956.25000,1.000000,2.250000e+05,-12539.000000,-410.000000,1.0,0.000000,1.000000,0.000000,3.000000,-12.000000
max,36456.00000,5.000000,1.575000e+06,-7489.000000,365243.000000,1.0,1.000000,1.000000,1.000000,7.000000,0.000000


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          10000 non-null  int64  
 1   gender         10000 non-null  object 
 2   car            10000 non-null  object 
 3   reality        10000 non-null  object 
 4   child_num      10000 non-null  int64  
 5   income_total   10000 non-null  float64
 6   income_type    10000 non-null  object 
 7   edu_type       10000 non-null  object 
 8   family_type    10000 non-null  object 
 9   house_type     10000 non-null  object 
 10  DAYS_BIRTH     10000 non-null  int64  
 11  DAYS_EMPLOYED  10000 non-null  int64  
 12  FLAG_MOBIL     10000 non-null  int64  
 13  work_phone     10000 non-null  int64  
 14  phone          10000 non-null  int64  
 15  email          10000 non-null  int64  
 16  occyp_type     6848 non-null   object 
 17  family_size    10000 non-null  float64
 18  begin_m

In [10]:
# 불필요한 컬럼 제거
data.drop(columns=['index', 'credit'], inplace=True)
test.drop(columns=['index'],         inplace=True)

In [11]:
cat_columns = [c for c, t in zip(data.dtypes.index, data.dtypes) if t == 'O'] 
num_columns = [c for c    in data.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n'.format(cat_columns))
print('Numeric Columns: \n{}'.format(num_columns))

Categorical Columns: 
['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']

Numeric Columns: 
['child_num', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'family_size', 'begin_month']


#### 라벨 데이터 인코딩

In [12]:
label = label.astype(int)

#### 전처리 프로세스 함수로 작성

In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [14]:
def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()
    
    tmp_x_train.reset_index(drop=True, inplace=True)
    tmp_x_valid.reset_index(drop=True, inplace=True)
    
    # 결측치 처리 + 수치형 범주형 결측치처리
    mice = IterativeImputer(random_state = 123)
    tmp_x_train[num_columns] = mice.fit_transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = mice.transform(tmp_x_valid[num_columns])
    tmp_x_valid[num_columns] = mice.transform(tmp_x_valid[num_columns])
    
    cat_imputer = SimpleImputer(strategy='most_frequent').fit(tmp_x_train[cat_columns])
    tmp_x_train[cat_columns] = cat_imputer.transform(tmp_x_train[cat_columns])
    tmp_x_valid[cat_columns] = cat_imputer.transform(tmp_x_valid[cat_columns])
    tmp_x_test[cat_columns] = cat_imputer.transform(tmp_x_test[cat_columns])
    
    # 스케일링 + Standard Scaler
    std_sca = StandardScaler().fit(tmp_x_train[num_columns])
    tmp_x_train[num_columns] = std_sca.transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = std_sca.transform(tmp_x_valid[num_columns])
    tmp_x_test[num_columns] = std_sca.transform(tmp_x_test[num_columns])
    
    # 인코딩 + One Hot Encoder
    ohe = OneHotEncoder(sparse = False).fit(tmp_x_train[cat_columns])
    tmp_x_train_cat = pd.DataFrame(ohe.transform(tmp_x_train[cat_columns]))
    tmp_x_valid_cat = pd.DataFrame(ohe.transform(tmp_x_valid[cat_columns]))
    tmp_x_test_cat = pd.DataFrame(ohe.transform(tmp_x_test[cat_columns]))
    
    tmp_x_train.drop(columns=cat_columns, inplace=True)
    tmp_x_valid.drop(columns=cat_columns, inplace=True)
    tmp_x_test.drop(columns=cat_columns, inplace=True)
    
    tmp_x_train = pd.concat([tmp_x_train, tmp_x_train_cat], axis=1)
    tmp_x_valid = pd.concat([tmp_x_valid, tmp_x_valid_cat], axis=1)
    tmp_x_test  = pd.concat([tmp_x_test, tmp_x_test_cat], axis=1)
    
    return tmp_x_train.values, tmp_x_valid.values, tmp_x_test.values

## Ensemble
개인적으로 앙상블은 머신러닝의 꽃이라고 생각합니다. 단일 모델로 좋은 성능을 이끄는 것도 중요하지만, 서로 다른 모델의 다양성을 고려하여 결과를 이끌어내는 앙상블은 응용할 수 있는 방법이 매우 많습니다. <br>
그 중 대표적인 2가지 앙상블에 대해 실습하고 배워보도록 하겠습니다. 

### 1. Voting Ensemble
이름에서 알 수 있듯이 각자의 모델이 투표를 하여 클래스를 선택하는 방식의 앙상블 입니다. <br>
Voting 앙상블은 Sklearn 자체적으로 모델로써 지원을 하며, 사용하기도 매우 쉽습니다. <br>
그리고 Hard, Soft로 Voting 방식이 나뉘는데, Hard는 라벨 값으로 투표를 하는 방식이고, Soft는 확률 값을 모두 더해 가장 높은 클래스를 선택합니다.

Voting Classifier는 Sklearn의 ensemble 패키지에 있습니다.

In [15]:
from sklearn.model_selection import train_test_split

# 쪼개어진 Train, Valid 데이터의 비율은 (7:3), 내부 난수 값 42, 데이터를 쪼갤 때 섞으며 label 값으로 Stratify 하는 코드 입니다. random_state를 주석 처리하고 데이터를 확인해보시면 계속 바뀝니다.
x_train, x_valid, y_train, y_valid = train_test_split(data, label, 
                                                      test_size=0.3,
                                                      random_state=42,
                                                      shuffle=True,
                                                      stratify=label)

In [16]:
x_train, x_valid, _ = preprocess(x_train, x_valid, test)

#### 1) 모델 불러오기 및 정의하기

In [17]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

clfs = [['Logistic', LogisticRegression()],
        ['RandomForest', RandomForestClassifier()],
        ['MLP', MLPClassifier()]]

vote_clf = VotingClassifier(clfs, voting='soft', n_jobs=4)

#### 2) 모델 학습하기

In [18]:
# 여기서 x_train, y_train은 마지막 Fold
vote_clf.fit(x_train, y_train)

VotingClassifier(estimators=[['Logistic', LogisticRegression()],
                             ['RandomForest', RandomForestClassifier()],
                             ['MLP', MLPClassifier()]],
                 n_jobs=4, voting='soft')

#### 3) 결과 확인하기

In [19]:
from sklearn.metrics import f1_score, log_loss

print('Validation F1 score : {:.4f}'.format(f1_score(y_valid, vote_clf.predict(x_valid), average='weighted')))

Validation F1 score : 0.6492


In [20]:
# voting hard 에서는 작동 x, 확률로 값을 뽑을 수 없음.
print('Validation log_loss score : {:.4f}'.format(log_loss(y_valid, vote_clf.predict_proba(x_valid))))

Validation log_loss score : 0.7484


### 2. Out-of-fold(OOF) Ensemble (같이 푸는 실습)
OOF 앙상블은 KFold 교차 검증에서 생성되는 각 Fold에 대한 예측 값을 앙상블하는 기법으로 모델 검증과 함께 앙상블을 진행할 수 있다는 장점이 있습니다. <br>

Cross Validation 파트에서 배웠던 KFold 코드를 재사용해 OOF 앙상블을 진행해보겠습니다.

In [21]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

val_scores = list()
oof_pred = np.zeros((test.shape[0], 3))
n_splits=5
    
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
    x_train, y_train = data.iloc[trn_idx, :], label.iloc[trn_idx,]
    x_valid, y_valid = data.iloc[val_idx, :], label.iloc[val_idx,]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    model = XGBClassifier(n_estimators = 10000,
                          n_jobs = n_cpus - 1)
    
    
    # 모델 학습
    model.fit(x_train, y_train, 
              eval_set = [[x_train, y_train], [x_valid, y_valid]],
              eval_metric = 'mlogloss',
              early_stopping_rounds = 100,
              verbose = 100)
    
    # 훈련, 검증 데이터 log_loss 확인
    trn_logloss = log_loss(y_train, model.predict_proba(x_train))
    val_logloss = log_loss(y_valid, model.predict_proba(x_valid))
    print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))
    
    val_scores.append(val_logloss)
    
    oof_pred += model.predict_proba(x_test) / skf.n_splits 

# 교차 검증 정확도 평균 계산하기
print('Cross Validation Score : {:.4f}'.format(np.mean(val_scores)))

[0]	validation_0-mlogloss:0.97461	validation_1-mlogloss:0.97771
[100]	validation_0-mlogloss:0.57168	validation_1-mlogloss:0.73423
[200]	validation_0-mlogloss:0.45537	validation_1-mlogloss:0.72717
[264]	validation_0-mlogloss:0.40251	validation_1-mlogloss:0.73030
0 Fold, train logloss : 0.48914, validation logloss : 0.7256
[0]	validation_0-mlogloss:0.97306	validation_1-mlogloss:0.97917
[100]	validation_0-mlogloss:0.55875	validation_1-mlogloss:0.74601
[200]	validation_0-mlogloss:0.44718	validation_1-mlogloss:0.73929
[261]	validation_0-mlogloss:0.39835	validation_1-mlogloss:0.74403
1 Fold, train logloss : 0.48454, validation logloss : 0.7386
[0]	validation_0-mlogloss:0.97445	validation_1-mlogloss:0.97846
[100]	validation_0-mlogloss:0.56397	validation_1-mlogloss:0.74330
[200]	validation_0-mlogloss:0.44434	validation_1-mlogloss:0.73880
[253]	validation_0-mlogloss:0.40128	validation_1-mlogloss:0.74246
2 Fold, train logloss : 0.49234, validation logloss : 0.7368
[0]	validation_0-mlogloss:0.973

In [22]:
submit_path = join(BASE_DIR, 'data', 'MDC14', 'sample_submission.csv')

submit = pd.read_csv(submit_path)
submit

,index,0,1,2
0,26457,0,0,0
1,26458,0,0,0
2,26459,0,0,0
3,26460,0,0,0
4,26461,0,0,0
...,...,...,...,...
9995,36452,0,0,0
9996,36453,0,0,0
9997,36454,0,0,0
9998,36455,0,0,0


In [23]:
submit.iloc[:, 1:] = oof_pred
submit

,index,0,1,2
0,26457,0.065646,0.110138,0.824216
1,26458,0.196105,0.169436,0.634459
2,26459,0.090482,0.136818,0.772700
3,26460,0.143019,0.111345,0.745636
4,26461,0.075181,0.086814,0.838005
...,...,...,...,...
9995,36452,0.178433,0.240356,0.581210
9996,36453,0.156201,0.338527,0.505272
9997,36454,0.015781,0.038529,0.945690
9998,36455,0.080006,0.243895,0.676099


## Two Stage Ensemble
### Stacking
- 지난 수업에 kFold를 활용한 OOF 앙상블에 대해 학습했습니다. Stacking은 말 그대로 모델의 결과를 쌓아서 앙상블을 하는 방식입니다.
- 스태킹의 원리는, 모델이 예측한 y 값은 "실제 y 값과 매우 선형성이 높다는 점을 이용하여 y_train_pred를 변수로 사용합니다.
- y_train_pred는 모든 vaild fold의 예측 값을 합쳐서 만듭니다.

1. 우선 서로 다른 모델에 대해 y_train_pred, y_test_pred 값을 모읍니다.
2. 모은 y_train_pred 값들을 axis=1 방향으로 합친 데이터를 new_x_train이라고 하겠습니다. 
    - 당연히 y_test_pred 값들도 axis=1 방향으로 합쳐 new_x_test라고 합니다.
3. new_x_train 데이터와, y_train 데이터로 모델을 학습하고, new_x_test로 최종 y_test_pred를 예측합니다.
    - 이때 주로 2 stage 모델(meta model이라고도 부릅니다.)은 성능이 강력한 모델을 사용합니다. (사실 해봐야 xgb, lgb 입니다..)

In [24]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

val_scores = list()
# 결과 값들을 stacking 해야하기 때문에, (모델 개수, 샘플의 수, 3) 라는 차원으로 구성됩니다.
oof_train = np.zeros((6, data.shape[0], 3))
oof_test  = np.zeros((6, test.shape[0], 3))

# oof_pred = np.zeros((test.shape[0], 3))

for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
    x_train, y_train = data.iloc[trn_idx, :], label.iloc[trn_idx,]
    x_valid, y_valid = data.iloc[val_idx, :], label.iloc[val_idx,]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    models = [RandomForestClassifier(n_estimators=200, max_depth=5,  random_state=42, n_jobs=(n_cpus-1)),
              RandomForestClassifier(n_estimators=150, max_depth=8, random_state=42, n_jobs=(n_cpus-1)),
              XGBClassifier(n_estimators=200, max_depth=5, random_state=42, n_jobs=(n_cpus-1)),
              XGBClassifier(n_estimators=150, max_depth=8, random_state=42, n_jobs=(n_cpus-1)),
              LGBMClassifier(n_estimators=200, max_depth=5, random_state=42, n_jobs=(n_cpus-1)),
              LGBMClassifier(n_estimators=150, max_depth=8, random_state=42, n_jobs=(n_cpus-1))]
            
    for j, model in enumerate(models):
        # 모델 학습
        model.fit(x_train, y_train)

        # j번째 칸에 맞는 결과 담기.
        oof_train[j, val_idx,] += model.predict_proba(x_valid)
        oof_test[j, :,]        += model.predict_proba(x_test) / n_splits
    
    print(f'{i} Fold, ...')

[17:48:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:48:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0 Fold, ...
[17:49:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:49:09] WARNING: C:/Users/Administrator/workspace/xg

In [25]:
oof_train.T.shape

(3, 26457, 6)

In [26]:
oof_pred.T.shape

(3, 10000)

In [27]:
pd.DataFrame(np.concatenate(oof_train.T, axis=1)).shape

(26457, 18)

In [28]:
# 모은 train, test의 예측 값을 new_x_train, new_x_test로 사용합니다.
new_train = pd.DataFrame(np.concatenate(oof_train, axis=1))
new_test  = pd.DataFrame(np.concatenate(oof_test))

In [29]:
new_train.shape, new_test.shape

((26457, 18), (60000, 3))

#### OOF 앙상블을 진행합니다.

In [ ]:
from sklearn.preprocessing import StandardScaler

val_scores = list()
oof_pred  = np.zeros((test.shape[0], 3))

for i, (trn_idx, val_idx) in enumerate(skf.split(new_train, label)):
    x_train, y_train = new_train.iloc[trn_idx, :], label[trn_idx]
    x_valid, y_valid = new_train.iloc[val_idx, :], label[val_idx]
    
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train     = scaler.transform(x_train)
    x_valid     = scaler.transform(x_valid)
    new_x_test  = scaler.transform(new_test)

    # 모델 정의
    model = XGBClassifier(random_state=42, n_jobs=(n_cpus-1))
    
    # 모델 학습
    model.fit(x_train, y_train)

    # 훈련, 검증 데이터 log_loss 확인
    trn_logloss = log_loss(y_train, model.predict_proba(x_train))
    val_logloss = log_loss(y_valid, model.predict_proba(x_valid))
    print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))
    
    val_scores.append(val_logloss)
    
    # 반드시 log의 역함수인 exp를 취해주세요.
    oof_pred += model.predict_proba(new_x_test) / n_splits
    

# 교차 검증 log loss 평균 계산하기
print('Cross Validation Score : {:.5f}'.format(np.mean(val_scores)))

In [ ]:
submit.iloc[:, 1:] = oof_pred
submit

In [ ]:
# submit.to_csv('stacking_first_submit.csv', index=False)

## 실습 솔루션

### 전처리 함수 만들기

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()
    
    tmp_x_train.reset_index(drop=True, inplace=True)
    tmp_x_valid.reset_index(drop=True, inplace=True)
    
    # 결측치 처리
    imputer = SimpleImputer(strategy='most_frequent')
    tmp_x_train[cat_columns] = imputer.fit_transform(tmp_x_train[cat_columns])
    tmp_x_valid[cat_columns] = imputer.transform(tmp_x_valid[cat_columns])
    tmp_x_test[cat_columns]  = imputer.transform(tmp_x_test[cat_columns])
    
    # 스케일링
    scaler = StandardScaler()
    tmp_x_train[num_columns] = scaler.fit_transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = scaler.transform(tmp_x_valid[num_columns])
    tmp_x_test[num_columns]  = scaler.transform(tmp_x_test[num_columns])

    # 인코딩
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(tmp_x_train[cat_columns])
    
    tmp_x_train_cat = pd.DataFrame(ohe.transform(tmp_x_train[cat_columns]))
    tmp_x_valid_cat = pd.DataFrame(ohe.transform(tmp_x_valid[cat_columns]))
    tmp_x_test_cat  = pd.DataFrame(ohe.transform(tmp_x_test[cat_columns]))
    
    tmp_x_train.drop(columns=cat_columns, inplace=True)
    tmp_x_valid.drop(columns=cat_columns, inplace=True)
    tmp_x_test.drop(columns=cat_columns, inplace=True)
    
    tmp_x_train = pd.concat([tmp_x_train, tmp_x_train_cat], axis=1)
    tmp_x_valid = pd.concat([tmp_x_valid, tmp_x_valid_cat], axis=1)
    tmp_x_test  = pd.concat([tmp_x_test, tmp_x_test_cat], axis=1)
    
    return tmp_x_train, tmp_x_valid, tmp_x_test

### Out-of-fold ensemble 실습 

In [ ]:
from sklearn.model_selection import StratifiedKFold

val_scores = list()
oof_pred = np.zeros((test.shape[0], 3))
n_splits = 5

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
    x_train, y_train = data.iloc[trn_idx, :], label.iloc[trn_idx,]
    x_valid, y_valid = data.iloc[val_idx, :], label.iloc[val_idx,]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    
    # 모델 학습
    model.fit(x_train, y_train)

    # 훈련, 검증 데이터 log_loss 확인
    trn_logloss = log_loss(y_train, model.predict_proba(x_train))
    val_logloss = log_loss(y_valid, model.predict_proba(x_valid))
    print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))
    
    val_scores.append(val_logloss)
    
    oof_pred += model.predict_proba(x_test) / skf.n_splits 

# 교차 검증 정확도 평균 계산하기
print('Cross Validation Score : {:.4f}'.format(np.mean(val_scores)))